In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 125

import seaborn as sns

import pandas as pd
import numpy as np
np.random.seed(0) 

import regex as re
import time 
from tqdm.notebook import tqdm 
tqdm.pandas()

import requests
from bs4 import BeautifulSoup


In [2]:
# Test the first url
url = "https://nltimes.nl/?page=1"

html_text = requests.get(url).text
html_tree = BeautifulSoup(html_text, 'html.parser')
news_items = html_tree.findAll('div', class_='news-card col-lg-4 col-sm-6 col-xs-12')

# Get link, title, time and category from news_items
raw_data = pd.DataFrame(columns=["links","title","time","content","class"])
for item in news_items:
    title = item.find('div', class_='news-card__title').text.replace('\n','')
    time = item.find('div', class_='news-card__date').text
    link = item.find('a').attrs["href"]
    category = item.find('div', class_='news-card__categories').text
    raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)

raw_data.head()
    

C:\Users\thanh\AppData\Local\Temp\ipykernel_31692\2762706346.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_31692\2762706346.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_31692\2762706346.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_31692\2762706346.py:15:

,links,title,time,content,class
0,/2023/01/11/work-home-thursday-strong-wind-hea...,Work from home on Thursday: Strong wind and he...,11 January 2023 - 13:17,NaN,TechNature
1,/2023/01/11/labor-strikes-trash-collectors-civ...,"Labor strikes: Trash collectors, civil servant...",11 January 2023 - 13:05,NaN,Business
2,/2023/01/11/security-tightened-serious-threat-...,Security tightened after “serious threat” from...,11 January 2023 - 12:17,NaN,Crime
3,https://nltimes.nl/2023/01/10/netherlands-resi...,Netherlands residents can apply for subsidy to...,10 January 2023 - 14:50,NaN,Business
4,https://nltimes.nl/2023/01/10/eu87-million-pus...,€87 million pushed into AI research in Amsterdam,10 January 2023 - 14:00,NaN,TechInnovation


In [3]:
raw_data  = raw_data .drop(raw_data .index[range(3)])


In [4]:
for _, row in raw_data.iterrows():
    news_page = requests.get(row["links"]).content
    news_tree = BeautifulSoup(news_page, "html.parser")
 
    # Export content
    try:
        content = news_tree.find('body').find_all('p')
        unwanted = ['Reporting by ANP', '© 2012-2023, NL Times, All rights reserved.']
        for x in list(dict.fromkeys(content)):
            if x.text.strip() not in unwanted:
                row["content"] += x.text
    except:
        row["content"] = ''
    
raw_data.head()

,links,title,time,content,class
3,https://nltimes.nl/2023/01/10/netherlands-resi...,Netherlands residents can apply for subsidy to...,10 January 2023 - 14:50,,Business
4,https://nltimes.nl/2023/01/10/eu87-million-pus...,€87 million pushed into AI research in Amsterdam,10 January 2023 - 14:00,,TechInnovation
5,https://nltimes.nl/2023/01/10/shots-fired-van-...,Shots fired when van in suspicious Amsterdam i...,10 January 2023 - 13:12,,Crime
6,https://nltimes.nl/2023/01/10/much-less-cocain...,Much less cocaine intercepted at Rotterdam por...,10 January 2023 - 12:20,,Crime
7,https://nltimes.nl/2023/01/10/14-million-nl-re...,1.4 million NL residents switched health insur...,10 January 2023 - 11:30,,HealthBusiness
